<a href="https://colab.research.google.com/github/sean-halpin/ml_tweepy_proj/blob/main/twitter_sentiment_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tweepy --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAN3pZwEAAAAASCDCIM1RgmZs05yRpfRg2Tis6bQ%3D9kVCbniKGCirILObLj1cHCWCHMPlxbAsA05C8rI4Kx2lsS8s5a"

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

In [ ]:
API_KEY=""
API_SECRET=""
ACCESS_TOKEN=""
ACCESS_TOKEN_SECRET=""

In [ ]:
import tweepy as tpy
import tweepy as tweepy
import pandas as pd

client = tweepy.Client(
    bearer_token=BEARER_TOKEN
)

In [ ]:
search_result=[]
for tweet in tweepy.Paginator(client.search_recent_tweets, "elon musk -is:retweet",
                              max_results=100, user_fields=['id','username','location'], 
                              tweet_fields=['id','author_id','created_at','geo','public_metrics','source'],
                              place_fields=['place_type', 'geo', 'country'], expansions=['geo.place_id', 'author_id']).flatten(limit=3500):
  search_result.append(tweet)

In [ ]:
len(search_result)

In [ ]:
print(search_result[0].text)
print(search_result[0].created_at)
print(search_result[0].geo)
print(search_result[0].public_metrics)
print(search_result[0].id)
print(search_result[0].author_id)
print(search_result[0].source)
print(search_result[0])
print(dir(search_result[0]))


In [ ]:
!pip install langdetect

In [ ]:
from langdetect import detect

def classify_lang(input_text):
  result = None
  try:
    result = detect(input_text)
  except:
    print("An exception occurred, classify_lang")
  return result

In [ ]:
list_of_lists = []
i=0
for sr in search_result:
  print(i)
  i+=1
  if (classify_lang(sr.text) == "en"):
    text = sr.text
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    list_of_lists.append([sr.id,sr.author_id,sr.text,sr.geo, sr.created_at, sr.source,str(labels[ranking[0]])])

1065
1066
1067
1068
1069
1070
1071
1072
1073
1074
1075
1076
1077
1078
1079
1080
An exception occurred, classify_lang
1081
1082
1083
1084
1085
1086
1087
1088
1089
1090
1091
1092
1093
1094
1095
1096
1097
1098
1099
1100
1101
1102
1103
1104
1105
1106
1107
1108
1109
1110
1111
1112
1113
1114


In [ ]:
cols = ["id","author_id","tweet","geo","created_at","source","sentiment"]
df = pd.DataFrame(list_of_lists, columns=cols)
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
df = df.replace(r'\r+|\n+|\t+',' ', regex=True)

In [ ]:
df.isnull().any()

In [ ]:
import time
epoch_time = int(time.time())

In [ ]:
df.to_csv("tweets_annotated.{}.csv".format(epoch_time), header=cols, index=False, escapechar="\\", encoding='utf-8')

In [ ]:
test_df = pd.read_csv("tweets_annotated.{}.csv".format(epoch_time))

In [ ]:
test_df.head(5)

In [ ]:
test_df['sentiment']. value_counts()

In [ ]:
test_df.info()